# Baseline DTW Evaluation

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn import metrics


## Load pair metadata and cached DTW distances

In [ ]:

# paths
pairs_path = Path('results/pairs_meta.parquet')
dtw_cache_path = Path('results/dtw_cache.parquet')

pairs = pd.read_parquet(pairs_path)
dtw = pd.read_parquet(dtw_cache_path)

# merge on pair index
pairs = pairs.reset_index().rename(columns={'index':'pair_id'})
merged = pairs.merge(dtw, on='pair_id')
merged.head()


## ROC and DET curves

In [ ]:

# compute fpr, tpr for a range of thresholds using raw and normalised distances
scores = {
    'raw': merged['d_raw'],
    'norm1': merged['d_norm1'],
    'norm2': merged['d_norm2'],
}
labels = merged['label']

roc_data = {}
auc_scores = {}
for key, vals in scores.items():
    fpr, tpr, thr = metrics.roc_curve(labels, -vals)
    roc_data[key] = (fpr, tpr, thr)
    auc_scores[key] = metrics.auc(fpr, tpr)


In [ ]:

fig, ax = plt.subplots(figsize=(4,4))
for key,(fpr,tpr,thr) in roc_data.items():
    ax.plot(fpr,tpr,label=f'{key} (AUC={auc_scores[key]:.3f})')
ax.plot([0,1],[0,1],'k--',lw=1)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.legend()
fig.tight_layout()
fig_path = Path('figures/roc_curve.png')
fig_path.parent.mkdir(exist_ok=True, parents=True)
fig.savefig(fig_path)
fig_path


In [ ]:

# DET plot
from matplotlib import pyplot as plt
fig, ax = plt.subplots(figsize=(4,4))
for key,(fpr,tpr,thr) in roc_data.items():
    fnr = 1 - tpr
    ax.plot(metrics.det_curve(labels, -scores[key])[0], metrics.det_curve(labels,-scores[key])[1], label=key)
ax.set_xlabel('False Positive Rate (log)')
ax.set_ylabel('False Negative Rate (log)')
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()
fig.tight_layout()
fig_path = Path('figures/det_curve.png')
fig.savefig(fig_path)
fig_path


## Equal Error Rate

In [ ]:

from scipy.interpolate import interp1d

def compute_eer(fpr, tpr):
    fnr = 1 - tpr
    abs_diffs = np.abs(fpr - fnr)
    idx = np.argmin(abs_diffs)
    return fpr[idx]

metrics_table = []
for key,(fpr,tpr,thr) in roc_data.items():
    eer = compute_eer(fpr, tpr)
    metrics_table.append({'metric':key,'EER':eer,'AUC':auc_scores[key]})
metrics_df = pd.DataFrame(metrics_table)
metrics_df


## Threshold analysis

In [ ]:

# Global threshold using d_norm1 by default
thresholds = {}
for key,(fpr,tpr,thr) in roc_data.items():
    idx = np.argmin(np.abs(fpr - (1-tpr)))
    thresholds[key] = thr[idx]
thresholds


In [ ]:

# Per-user thresholds based on d_norm1
per_user = merged.groupby('userA')
user_thresholds = {}
for user, group in per_user:
    fpr, tpr, thr = metrics.roc_curve(group['label'], -group['d_norm1'])
    idx = np.argmin(np.abs(fpr - (1-tpr)))
    user_thresholds[user] = thr[idx]
plt.hist(list(user_thresholds.values()), bins=20)
plt.xlabel('Threshold')
plt.ylabel('Count')
fig_path = Path('figures/threshold_dist.png')
plt.tight_layout()
plt.savefig(fig_path)
fig_path


## Save metrics

In [ ]:

results_path = Path('results/baseline_metrics.csv')
results_path.parent.mkdir(exist_ok=True, parents=True)
metrics_df.to_csv(results_path, index=False)
results_path
